# Acknowledgement
### This Notebook has been realised owing to the tutorial series by Valerio Velardo. The playlist can be found at https://www.youtube.com/channel/UCZPFjMe1uRSirmSpznqvJfQ/playlists under the title "Deep Learning (for Audio) with python. His tutorial involved genre-classification among 10 classes. 
I modified his work for my mood-classification among 4 classes task. However, the deep learning architecture used was not influenced by his tutorial. His main influence has been in user defined functions for saving and loading data with preprocessing the data, and ploting the history curve. Valerio has been made aware of me using his tutorial for my project through LinkedIn. He even undertook my survey and had like a post regarding my project on LinkedIn

# Load Google Drive(s)

In [ ]:
## Use the account 
## Email: arvind.final.dissertation@gmail.com
## Password: final paper

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
# !sudo mkdir /content/drive1
# !google-drive-ocamlfuse /content/drive1

# Libraries

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plot
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers import Conv2D, MaxPooling2D, Input, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.regularizers import l2

#Sci-kit learn libs
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

# User Defined Function

In [ ]:
def json_load(json_path):
    """Loads the pre-processessed json file.

        json_path: Path to json file containing data
        X : MFFC array
        y : encoded labels
        mapping: corresponding labels
    """

    with open(json_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    mapping = data["mapping"]
    return X, y, mapping

In [ ]:
def viz_history(history):
    """Plots accuracy-vs-epoch and error-vs-epoch plots with 

        :param history: Training history of model
        :return:
    """

    figure, axes = plot.subplots(figsize=(10,4), nrows=1, ncols=2)
    figure.tight_layout(pad=3.0)
    
    axes[0].set_xlabel("Epoch")
    axes[1].set_xlabel("Epoch")

    # create accuracy sublpot
    axes[0].plot(history.history["accuracy"], label="Train Accuracy")
    axes[0].plot(history.history["val_accuracy"], label="Validation Accuracy")
    axes[0].set_ylabel("Accuracy")
    axes[0].legend(loc="upper right")
    axes[0].set_title("Accuracy-vs-Epoch")

    # create error sublpot
    axes[1].plot(history.history["loss"], label="Train Error")
    axes[1].plot(history.history["val_loss"], label="Validation Error")
    axes[1].set_ylabel("Error")    
    axes[1].legend(loc="upper right")
    axes[1].set_title("error-vs-epoch")

    plot.show()


In [ ]:
def train_val_test(validation_split, test_split):
    """Loads data and splits it into train, validation and test sets.

        In AER splits, Validation_split of 0.1 and test_split of 0.1 is used ----> train:0.8,val:0.09,test:0.1
        The above split produced the highest accuracies
    """

    # load data
    X, y, mapping = json_load(JSON_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split, random_state=42, stratify = y)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_split, random_state=42, stratify = y_train)

    return X_train, X_validation, X_test, y_train, y_validation, y_test, mapping

In [ ]:
def build_model_LSTM(input_shape):
  """Generates RNN-LSTM model

  :param input_shape (tuple): Shape of input set
  :return model: RNN-LSTM model
  """
  reg = 0.0001
  def mish(x):
   return x * keras.backend.tanh(keras.backend.softplus(x))

  # build network topology
  model = keras.Sequential()

  # 2 LSTM layers
  model.add(keras.layers.Bidirectional(keras.layers.LSTM(194, input_shape=input_shape, return_sequences=True )))
  model.add(keras.layers.Dropout(0.7))
  model.add(keras.layers.Bidirectional(keras.layers.LSTM(194)))
  model.add(keras.layers.Dropout(0.7))

  # dense layer
  model.add(keras.layers.Dense(512, activation= mish, bias_regularizer = l2(reg)))
  model.add(keras.layers.Dense(512, activation= mish, bias_regularizer = l2(reg)))
  model.add(keras.layers.Dense(128, activation= mish,  bias_regularizer = l2 (reg)))
  
  # output layer
  model.add(keras.layers.Dense(4, activation='softmax'))

  return model

In [ ]:
def load_model(model_path):
      """Loads model from the given path
        :param model_path: route to the desired model
        :return model: return the loaded model
      """
      
      def mish(x):
        return x * keras.backend.tanh(keras.backend.softplus(x))

      model= keras.models.load_model(model_path,
                        custom_objects = {"mish":mish})

      return model

In [ ]:
def predict_AER(X_predict):
    """Gives the AER prediction of the loaded model.
      
      :param X_predict: the instance to predict
    """
    X = X_predict
    X= X[np.newaxis, ...] # array shape (1, 130, 13, 1)
    label_dict = {}
    for i in range(len(mapping)):
      label_dict.update({i:mapping[i]})
    label_dict

    # perform prediction
    prediction = model.predict(X)

    # get index with max value
    predicted_index = np.argmax(prediction, axis=1)
    print("The predicted mood is:", label_dict[predicted_index[0]])

In [ ]:
def metrics_all(model, X_test, y_test):
    """Provides different metrices to assess.

      :param X_test: The set of instances to predict on
      :param Y_test: Ground truth labels
    """

    yhat_probs = model.predict(X_test, verbose=0) #class probability prediction
    yhat_classes = np.argmax(yhat_probs, axis =1) #predicted class


    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)

    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes, average='weighted')
    print('Precision: %f' % precision)

    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes, average='weighted')
    print('Recall: %f' % recall)

    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes, average='weighted')
    print('F1 score: %f' % f1)

    # Confusion matrix
    matrix = confusion_matrix(y_test, yhat_classes)
    print("\nThe confusion matrix is as below:")
    print(matrix)

In [ ]:
def accuracy_per_class(preds, labels):
    """
    Provides the accuracy for each class in the AER task

    :param preds: Predicted labels
    :param labels: Ground truth labels
    """

    label_dict = {} #dictionary to map the encoded labels to their semantic labels
    for i in range(len(mapping)):
      label_dict.update({i:mapping[i]})
      
    for label in np.unique(y_test):
        y_preds = yhat_classes[y_test==label]
        y_true = y_test[y_test==label]
        print(f'Class: {label_dict[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
        print(f'Accuracy (Percentage): ', (len(y_preds[y_preds==label])/len(y_true))*100, "%\n")

# Load and Split dataset

In [ ]:
# get train, validation, test splits
JSON_PATH = "/content/drive/My Drive/DEAM JSON/Sample rate:44100, MFCC coefficient:13, Track Duration:45, segments: 20, N0. FFT = 2048"
X_train, X_validation, X_test, y_train, y_validation, y_test, mapping = train_val_test(0.1, 0.1)

# Model Training

In [ ]:
# create network
shape_of_input = (X_train.shape[1], X_train.shape[2], 1)
model = build_model_LSTM(shape_of_input)

checkpoint = ModelCheckpoint('/content/drive/My Drive/Good Models/Model-best-model-accuracy-BiLSTM.h5', verbose = 1, monitor='val_accuracy',save_best_only=True, mode='max')
patience = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)

# initial_learning_rate = 0.00005
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=10000,
#     decay_rate=0.98,
#     staircase=True)

optimiser = tf.keras.optimizers.RMSprop(learning_rate= 0.00005)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

plot_model(model, to_file='/content/drive/My Drive/Good Models/BiLSTM_.png', show_shapes=True, show_layer_names=True)


# train model
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=64, epochs=200, callbacks=[checkpoint,patience])


Epoch 1/200
428/428 [==============================] - ETA: 0s - loss: 1.0732 - accuracy: 0.5919
Epoch 00001: val_accuracy improved from -inf to 0.60532, saving model to /content/drive/My Drive/Good Models/Model-best-model-accuracy-BiLSTM.h5
428/428 [==============================] - 95s 221ms/step - loss: 1.0732 - accuracy: 0.5919 - val_loss: 1.0486 - val_accuracy: 0.6053
Epoch 2/200
428/428 [==============================] - ETA: 0s - loss: 1.0203 - accuracy: 0.6161
Epoch 00002: val_accuracy improved from 0.60532 to 0.61683, saving model to /content/drive/My Drive/Good Models/Model-best-model-accuracy-BiLSTM.h5
428/428 [==============================] - 93s 218ms/step - loss: 1.0203 - accuracy: 0.6161 - val_loss: 1.0591 - val_accuracy: 0.6168
Epoch 3/200
428/428 [==============================] - ETA: 0s - loss: 0.9962 - accuracy: 0.6238
Epoch 00003: val_accuracy improved from 0.61683 to 0.62241, saving model to /content/drive/My Drive/Good Models/Model-best-model-accuracy-BiLSTM.h5



# Plots and Predictions (The Fun Part)

In [ ]:
viz_history(history)

NameError: ignored

In [ ]:
model = load_model("/content/drive/My Drive/Good Models/Model-best-model-accuracy-BiLSTM.h5")

In [ ]:
predict_AER(X_test[0])

The predicted mood is: Happy


In [ ]:
metrics_all(model,X_test,y_test)

Accuracy: 0.858959
Precision: 0.857037
Recall: 0.858959
F1 score: 0.857558

The confusion matrix is as below:
[[1048   63   38   33]
 [  41 1261   27   43]
 [  57   45  290   14]
 [  45   57   14  306]]


In [ ]:
yhat_probs = model.predict(X_test, verbose=0)
yhat_classes = np.argmax(yhat_probs, axis =1)

accuracy_per_class (yhat_classes, y_test)

Class: Happy
Accuracy: 1048/1182
Accuracy (Percentage):  88.66328257191202 %

Class: Sad
Accuracy: 1261/1372
Accuracy (Percentage):  91.90962099125365 %

Class: Angry
Accuracy: 290/406
Accuracy (Percentage):  71.42857142857143 %

Class: Relaxed
Accuracy: 306/422
Accuracy (Percentage):  72.51184834123224 %

